# Search AiiDA Database Slab optimizations

In [ ]:
%aiida
from apps.surfaces.widgets.search_completed_widget import SearchCompletedWidget

In [ ]:
PREPROC_VERSION = 1.10
s = SearchCompletedWidget(
    version=PREPROC_VERSION,
    wlabel='SlabGeoOptWorkChain',
    clabel='slab_geo_opt',
    fields_disable={})

display(s)